<a href="https://colab.research.google.com/github/w86w86/econ8320_US-Census/blob/main/MAIN_0511_1024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Section 2: Download all required files

###Call the API with my variables for my research

> List of some states "46 SD", "47 TN","10 DE", "2 AK","5 AR","24 MD","31 NE"

CBSA:

"46 SD",
* "43620 Sioux Falls, SD",

"47 TN"
* "16860 Chattanooga, TN-GA",
* "28940 Knoxville, TN",
* "34980": "Nashville-Davidson--Murfreesboro--Franklin, TN",
* "28700": "Kingsport-Bristol-Bristol, TN-VA",
* "32820": "Memphis, TN-MS-AR",
* "17300": "Clarksville, TN-KY",
* "27740": "Johnson City, TN",
* "17420": "Cleveland, TN",

"10 DE"
* "20100 Dover, DE",
* "428": "Philadelphia-Reading-Camden, PA-NJ-DE-MD",
* "41540": "Salisbury, MD-DE",
*
* 0 city for "2 AK"

"5 AR",
* "22900 Fort Smith, AR-OK",
* "30780 Little Rock-North Little Rock-Conway, AR",
* "22220 Fayetteville-Springdale-Rogers, AR-MO",
* "38220 Pine Bluff, AR",
* "32820 Memphis, TN-MS-AR",
*
"24 MD",
* "25180": "Hagerstown-Martinsburg, MD-WV",
* "428": "Philadelphia-Reading-Camden, PA-NJ-DE-MD",
* "12580": "Baltimore-Columbia-Towson, MD",
* "37980": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",
* "15680": "California-Lexington Park, MD",
* "41540": "Salisbury, MD-DE",

###LIBRARIES, CONSTANTES, GOOGLE DRIVE

In [ ]:
# (10) IMPORT LIBRARY
import requests
import os
import pandas as pd
import regex as re
import time
from datetime import datetime as dt
import json
import logging

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# (20) CONSTANTES

#Dict of all states
#all_states = requests.get('https://api.census.gov/data/2024/cps/basic/jan/variables.json')
#states = json.loads(all_states.text)
#states_dict = states['variables']['STATE']['values']['item']
#states = [state for state in states_dict.keys()]

#All year of the study


###(10) CLASS API

In [ ]:
# (30) CLASS API and Decorator [log_activity2]

def log_activity2(func):
    def wrapper(*args, **kwargs):
        # Create a logger
        logger = logging.getLogger('activity_logger')
        logger.setLevel(logging.INFO)

        # Create a file handler and set the logging format
        file_handler = logging.FileHandler('activity.log')
        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        file_handler.setFormatter(formatter)

        # Add the file handler to the logger
        logger.addHandler(file_handler)

        # Log the activity before calling the wrapped function
        logger.info(f'Activity: {func.__name__}')

        # Call the wrapped function
        return func(*args, **kwargs)

    return wrapper


class API:
  """This generic class API will handle at once all request to the CPS api
  How to?:
  > instanciate a variable at once, then use it to browse through all variables (not data) on the Census API json file
  > example api:    api=API()

  Properties:
  > allVars_dict (variable): a dict of all variables for each 'year' (key)
  > examples:
    - api.allVars_dict[2023]        * to se all the variables (dict) in 2023
    - 'GTCBSA' in api.allVars_dict[2024].keys() *                is <GTCBSA> in 2024?
    - api.allState_dict[2021]       * all stateID:description in year 2021"""

  def __init__(self, env='gdrive') -> None:
    """The var 'env' is to define where the program is running.
    Two possible values:
     - 'gdrive' : default
     - 'github'
    """
    self.df = pd.DataFrame() #this will contain final df to analyze
    self.get()
    self.state={}

    ##Read the ini varible that contains all necessary paths
    try:
      with open('vars.ini', "r") as varsini:
        vars_ini = json.load(varsini)
    except Exception as e:
      print('Please check vars.ini file')
      raise("Can find 'vars.ini', ini varible that contains all necessary paths ")

    self.vars={
        'columns': {
          'weight'   : {'def':'represents the information related to population counts','col':'weight'},
          'nativity' : {'def':'Demographics-native country of mother', 'col': 'PEMNTVTY'}, #list of countries
          'marital'  : {'def':'Demographics-marital status', 'col':'PEMARITL', 'val':{1:'Married', 2:'Not Married'}}, #1:'1,2', 2:'-1,3,4,5,6'},
          'sex'      : {'def':'Demographics-sex', 'col':'PESEX', 'val':{1:'Male', 2:'Female'}}, #no change
          'citiz'    : {'def':'Demographics-United States citizenship group', 'col':'PRCITSHP', 'val':{1:'Citizen', 2:'Not Citizen'}}, #1:'1,2,3,4', 2:'5'
          'occ1'     : {'def': 'Primary Job', 'col':'PTIO1OCD'}, #list of occupations
          'occ2'     : {'def':'Second Job', 'col':'PTIO2OCD'}, #list of occupations
          'collegcred':{'def':'college credit completed', 'val':{1:'Four or more years college credit?', 2:'Less than 4 years'}}, #1:'5' 2: others
          'highsch'     : {'def':'Second Job', 'col':'PEDIPGED', 'val':{1:'High School GED', 2:'No High School GED'}},  # 1:'1,2', 2:'-1'
          'city'     : {'def':'Demographics-city level', 'col':'GTCBSA'},
          'state'    : {'def':'Sate', 'col':'', 'val': json.loads(requests.get('https://api.census.gov/data/2024/cps/basic/jan/variables.json').text)['variables']['STATE']['values']['item']}
        },
        'new_census_data': False,
        'varini': vars_ini
    }

  def get(self):
    #TEST
    #abc = requests.get('https://api.census.gov/data/2023/cps/basic/jan/variables.json')
    #print(abc)
    self.allVars_dict = {year_: year_vars for year_, year_vars in \
            zip(self.allYearList(), [json.loads(requests.get('https://api.census.gov/data/'+ str(year_) +'/cps/basic/jan/variables.json').text)['variables'] for year_ in self.allYearList()])}

  @log_activity2
  def allStatesIdList(self, year:int=2010):
    return list(map(int, self.allVars_dict[year]['GESTFIPS']['values']['item'].keys())) #convert in int

  @log_activity2
  def allMonthList3TLA(self, specific_month=None):
    month_dict = {1:"jan",2:"feb",3:"mar",4:"apr",5:"may",6:"jun",7:"jul",8:"aug",9: "sep",10: "oct",11:"nov",12:"dec"}
    #return next(key for key, value in month_dict.items() if value == m)
    if specific_month is None:
      return list(month_dict.values())
    else:
      return [specific_month]

  def allYearList(self):
    return range(2010, 1+dt.now().year)

  @log_activity2
  def getVarsNameFromUrl(self, url):
    return re.findall(r'([A-Z]+)', url)

  def id_to_stateName(self,id):
    return self.vars['columns']['state']['val'][str(id)]

#api=API('gdrive')

###(20) CLASS FileCompMix:

In [ ]:
class FileCompMix:
  """This class will track the which State Year and Month has already been processed
  so we are ending up re-download the same file, knowing that we have close to 10,000 calls
  to make with an average of 2.50 secs = ~ 7 hours in total.

  Hou to?:
  > track = FileCompMix(filename [optional])
        - if filename contains any char then the class will generate a new fileName_%m%d-%H%M%.
        - defaut is   ' MixCompStatus.csv '
  > two main methods:
        - track.record_state_of_state_year_month()   *   to store the info
        - track.is_record_state_of_state_year_month  *   to check if elt exist already
  """
  def __init__(self, api) -> None:
    fileName  = '' #dt.now().strftime("_%m%d-%H%M%S")
    self.gdrivePath = api.vars['varini'][api.vars['varini']['env']]['unMergedPath']
    self.track_file = self.gdrivePath + 'MixCompStatus' + fileName + '.csv'
    self.set_df()
    self.eltInFileList = [list(i) for i in self.df.values]


  def set_df(self):
    """Read the NTFS file, and load it to a df so we can browse in in and download what has not been yet download
    """
    try:
      self.df = pd.read_csv(self.track_file, header=None)
    except Exception as e:
      self.df = pd.DataFrame([])
    self.hasCSVdata = self.df.shape[0]

  def numberOfelement(self):
    return self.df.shape[0]

  def record_state_of_state_year_month(self, element):
    self.df = pd.concat([self.df, pd.DataFrame([element])], ignore_index=True)
    #print (f'__record_state_of_state_year_month__ len: {self.df.shape[0]}')
    self.df.to_csv(self.track_file, mode='w', header=False, index=False) #not mode='a'
    self.hasCSVdata = True

  def is_record_state_of_state_year_month(self, element):
    if not self.hasCSVdata:
      #print ('--------------------------------#1F File emply, so will add')
      return False
    else:
      if (list(element) in self.eltInFileList):
        #print ('--------------------------------#2T elt already there, so not added, skipped')
        return True
      #print ('--------------------------------#3F, so will add')
      return False

###(40) CLASS MIXCOMP

In [ ]:
# (40) CLASS MIXCOMP
#Decorator to track API response time
def track_response_time(func):
    def wrapper(self, *args, **kwargs):
        start_time = time.time()
        result = func(self, *args, **kwargs)
        end_time = time.time()
        response_time = end_time - start_time
        self.response_time = response_time  # Store in the class
        print(f"Function: '{func.__name__}()'. Census API query took: {response_time} seconds")
        return result
    return wrapper


#Class for unit-request (All State, specific Year, specific Month)
class MixComp:
  """
This class will pull the Mix composition about the US population from one state.
Code	    Label
tabulate  Counts of instances
PEMNTVTY	Demographics-native country of mother
PEMARITL	Demographics-marital status
PESEX	    Demographics-sex
PRCITSHP	Demographics-United States citizenship group
GTCBSA	  Demographics-city level
STATE	    FIPS STATE Code
GTCBSA	  Demographics-city level

  """
  def __init__(self, api, year, month):
      #key = '&key='+'804d0a1a18d1de70764950c78e2a3a42d3d45e48' #w86
      key = '&key='+'a24a131b4da25b83199bdc7202fcab553b9e2983' #luos
      self.url = 'https://api.census.gov/data/'+str(year)+'/cps/basic/'+month+'?get=PWSSWGT,PEMNTVTY,PEMARITL,PESEX,PRCITSHP,PTIO1OCD,PTIO2OCD,PECYC,PEDIPGED,GTCBSA&for=state:*'+ key
      self.gdrivePath = api.vars['varini'][api.vars['varini']['env']]['unMergedPath']
      self.api = api
      self.activityLog = 'activity.log'
      self.year = year
      self.month = month
      self.response_time = 0 # how long the API took to run on Census CPS
      self.dataAPI = None
      self.dataText = None
      self.data4Df = []
      self.df = pd.DataFrame(None) #Will contain the final df(dataframe)
      self.initLogger()

  def initLogger(self):
      # Create a logger
      self.logger = logging.getLogger('activity_logger')
      self.logger.setLevel(logging.INFO)
      # Create a file handler and set the logging format
      file_handler = logging.FileHandler(self.activityLog)
      formatter = logging.Formatter('%(asctime)s - %(levelname)s - [w86] %(message)s')
      file_handler.setFormatter(formatter)
      # Add the file handler to the logger
      self.logger.addHandler(file_handler)

  def __str__(self):
      return f"""
      Mix Composition of the US population: month={self.month}, year={self.year}
      Dataframe head:
         {self.df.head(3)}"""

  def __repr__(self):
      return f"MixComp({self.year}, {self.month})"

  def __len__(self):
      return self.df.shape[0]

  def __add__(self, other):
      return pd.concat([self.df, other.df], axis=0)

  # to convert in class
  def checkVariable(self, year, url):
      """This function will check if a set of variables are declared on the Census API. Using the an url as input. Vars are in ALL_CAPS
      """
      matches = self.api.getVarsNameFromUrl(url)
      cps_vars = self.api.allVars_dict[year].keys()
      res = dict()
      for var in matches:
        res[var] = True if var in cps_vars else False
      res['status'] = all([status for status in res.values()])
      return res

  @track_response_time
  def get(self):
      #This section will validate all variables and replace the city's with the right one if year>=2024 (GTCBSA->CBSA)
      check = self.checkVariable(self.year, self.url)
      print (f"checkVariable() = {check}")
      if not check['status']:
        self.logger.info(f"> Class MixComp: / get() / BEFORE - Variables = {check}")
        if (self.year>=2024) and ('CBSA' not in check.keys()): #before 2024, City variable in 'GTCBSA', starting on 2024, City variable was 'CBSA'
          self.url = re.sub('GTCBSA', 'CBSA', self.url)
          self.logger.info(f"Class MixComp: / get() / > AFTER -  Variables = {self.checkVariable(self.year, self.url)}\nNew URL: {self.url}")
        else:
          self.logger.info(f"Class MixComp: / get() / > Please check all your Census variables. Unable to find some in the API call. Variables = {check}")

      try:
        # download the Demographics_Native_Country_Of_Mother for Mix-Compostion
        self.dataAPI = requests.get(self.url)
        res = dict({'response time in secs': round(self.response_time,2), 'status': True if 200==self.dataAPI.status_code else False, 'code':self.dataAPI.status_code, 'comment': 'url:' + self.url})
        self.logger.info(f'Class MixComp: / get() / > Return dict: [[[[ {res} ]]]]')
        return res

      except requests.exceptions.RequestException as e:
          # Handle connection errors or HTTP errors
          #raise("STOP ERROR: Tools for Data Science - Semester Project: An error occurred:\n [[[[", e, "]]]]")
          self.logger.info(f'Class MixComp: / get() / > STOP ERROR: Tools for Data Science - Semester Project: An error occurred:\n [[[[ {e} ]]]]')
          self.logger.info(f'Class MixComp: / get() / > Skipped !!')
      except AttributeError:
          self.logger.info(f'Class MixComp: / get() / > STOP ERROR: Please check your variable name!')

  def buildingDf(self):
      """
      ##Definition of variables:
      1. weight     represents the information related to population counts.
      2. nativity   column provides information about the state
      1. PEMNTVTY   Demographics-native country of mother
      3. marital    Demographics-marital status
      4. sex        Demographics-sex
      5. citiz      Demographics-United States citizenship group
      6. occ1       Primary Job
      7. occ2       Second Job
      8. collegcred college credit completed
      5. city       Demographics-city level
      We will understand the population composition by state about how mixed it is based on the native country of mothers,
      marital status distribution, gender demographics, citizenship status, at the city-level characteristics.
      This data can be used to uncover trends, patterns, and insights into the social and cultural fabric of the US population.
      """
      ##Build df and Renmane Df Columns for easy reading.
      self.df = pd.DataFrame(list(json.loads(self.dataAPI.text))[1:],columns = ['weight', 'nativity', 'marital', 'sex', 'citiz', 'occ1','occ2', 'collegcred','highsch', 'city', 'state'])
      ##Remove unknown City ID city==0
      self.df = self.df[self.df['city']!='0']
      self.logger.info(f'Class MixComp: / buildingDf > Remove unknown City ID, Build df and Renmane Df Columns for easy reading.')
      ##Building the YYYYMM column
      self.df['YYYYMM'] = dt.strptime(str(self.year)+ '-' +str(self.month), "%Y-%b")
      self.logger.info(f'Class MixComp: / buildingDf > Built Dataframe for YYYYMM:{str(self.year)}-{str(self.month)}')
      #convert column in true type 'citiz', 'occ1','occ2', 'collegcred','highsch', 'city', 'state'])
      self.df['weight'] = self.df['weight'].astype(float)
      self.df['nativity'] = self.df['nativity'].astype(int)
      self.df['marital'] = self.df['marital'].astype(int)
      self.df['sex'] = self.df['sex'].astype(int)
      self.df['citiz'] = self.df['citiz'].astype(int)
      self.df['occ1'] = self.df['occ1'].astype(int)
      self.df['occ2'] = self.df['occ2'].astype(int)
      self.df['collegcred'] = self.df['collegcred'].astype(int)
      self.df['highsch'] = self.df['highsch'].astype(int)
      self.df['city'] = self.df['city'].astype(int)
      self.df['state'] = self.df['state'].astype(int)
      #convert content with relevant information
      self.df['marital'] = self.df['marital'].apply(lambda x: 1 if x in [1,2] else 2)
      self.df['citiz'] = self.df['citiz'].apply(lambda x: 1 if x in [1,2,3,4] else 2)
      self.df['collegcred'] = self.df['collegcred'].apply(lambda x: 1 if x in [5] else 2)
      self.df['highsch'] = self.df['highsch'].apply(lambda x: 1 if x in [1,2] else 2)

  def save(self, file=''):
      try:
        if file == '':
          file = str(self.year)+'-'+ str(self.month)+'-MixComp.csv'
        r = self.df.to_csv( self.gdrivePath + file, index=False)
        self.logger.info(f'Class MixComp: / save > Created CSV file: {file}')
        return r
      except FileNotFoundError:
        raise (f'Check the path {file} of the file')
        self.logger.info(f'Class MixComp: / save > Check the path {file} of the file')
        self.logger.info(f'Class MixComp: / save > Skipped {file} creation')

  def main(self):
      self.logger.info(f"> {'v'*30} year:{self.year},month:{self.month} {'v'*30}")
      self.logger.info(f'Class MixComp: / main >  method - Entrance')
      self.logger.info(f'>Class MixComp: / main > get() method')

      url_call = self.get()
      self.logger.info(f'Class MixComp: / main - line: get() method - Result:{url_call}')

      try:
        if url_call['status']:
          self.buildingDf();  self.logger.info(f'Class MixComp: / main > URL found (200), now Creating the df')
          self.save();        self.logger.info(f'Class MixComp: / main > save() method')
        else:
          self.logger.info(f"Class MixComp: / main > Error during URL call, Code return: {url_call['code']}, Skipped !!")
      except Exception as e:
        self.logger.info(f"Class MixComp: / main > Error during URL call, Code return was undefinied, !! Please also check the API KEY!! Skipped !!")
      self.logger.info(f'Class MixComp: / main > Called main(self) method - End')
      self.logger.info(f"> {'^'*90}")
      return url_call


###(60#1) CORE #01 (this section will create individual files: yyyy-mm-MixComp.csv)

In [ ]:
## (60 - 1) Going through All Months at once

def single_file_generator_01(api):
  track = FileCompMix(api)
  count = {'initiallyInFile': track.numberOfelement(), 'processed': 0, 'skipped': 0}
  print ('Begin>',count)

  for year in api.allYearList():
    for month in api.allMonthList3TLA('jun'): #remove 'jun' for all months:
      elt = [year,month]
      if not track.is_record_state_of_state_year_month(elt):
        dataA = MixComp(api, *elt) #(year, month)

        result = dataA.main()
        count['processed'] += 1
        if result['status']:
          track.record_state_of_state_year_month(elt)
          api.vars['new_census_data'] = True
      else:
        count['skipped'] += 1

  print ('\nEnd  <',count)
  print (f"new_census_data: {api.vars['new_census_data']}, count['processed']: {count['processed']},count['skipped']: {count['skipped']}")

###(60#2) CORE #02 Merging to one (this section will create individual files: yyyy-Merged.csv)

In [ ]:
def merge_02(force=False):
    """Merge all single file in one and
    Will transfer all done file in a folder "mergedByYear"
    if force=True, the script will run whether we have new census data available
    or not -> api.vars['new_census_data']
    """
    if not (api.vars['new_census_data'] or force): return None
    filenameList = os.listdir(api.vars['varini'][api.vars['varini']['env']]['unMergedPath'])
    filenameList = [file for file in filenameList if re.match(r'^\d{4}', file)]

    print (filenameList)
    "Example: 2010-apr-MixComp.csv"

    for file in filenameList:
      year = file.split('-')[0]
      if (int(year)>=2010) and (int(year)<=2024):
        if year not in api.vars['varini'][api.vars['varini']['env']]['yearly']:
          api.vars['varini'][api.vars['varini']['env']]['yearly'][year]=[]
        df = pd.read_csv(api.vars['varini'][api.vars['varini']['env']]['unMergedPath'] + file)
        api.vars['varini'][api.vars['varini']['env']]['yearly'][year].append(df)

    api.vars['varini'][api.vars['varini']['env']]['fused'] = {}
    for year, dfs in api.vars['varini'][api.vars['varini']['env']]['yearly'].items():
      api.vars['varini'][api.vars['varini']['env']]['fused'][year] = pd.concat(dfs)

    for year, df in api.vars['varini'][api.vars['varini']['env']]['fused'].items():
      path = api.vars['varini'][api.vars['varini']['env']]['merged']
      df.to_csv(f'{path}{year}_merged.csv', index=False)


###(60#3) CORE #03 Group By using merged files (will generated new files in folder Merged2 | files: yyyy-Merged.csv)

In [ ]:
def group_by_03(force=False):
  """Read the merged file by year and group column in a different folder 'mergedByYear2'
  """
  if not (api.vars['new_census_data'] or force): return None
  filenameList = os.listdir(api.vars['varini'][api.vars['varini']['env']]['merged'])
  filenameList = [file for file in filenameList if re.match(r'^\d{4}', file)]
  """filenameList = iter(filenameList)
  file = next(filenameList)
  print (f'file:{file}')
  """
  for file in filenameList:
    print (f'file:{file}')
    df = pd.read_csv (api.vars['varini'][api.vars['varini']['env']]['merged'] + file)
    print (df.columns)

    grouped = df.groupby(['state', 'nativity', 'marital', 'sex', 'citiz', 'occ1', 'occ2',
                        'collegcred', 'highsch', 'city', 'YYYYMM'])['weight'].sum().reset_index()
    grouped.to_csv(f"{api.vars['varini'][api.vars['varini']['env']]['merged2']}{file}")


###(70) #04 Single df for all years

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
def readCSV_and_fuse_dfs_04():
  """read all files in a single df
  """
  filenameList = os.listdir(api.vars['varini'][api.vars['varini']['env']]['merged2'])
  filenameList = [file for file in filenameList if re.match(r'^\d{4}', file)]
  list_df=[]
  for file in filenameList:
    df = pd.read_csv (api.vars['varini'][api.vars['varini']['env']]['merged2'] + file)
    list_df.append(df)
  df = pd.concat(list_df)
  return df
  #print (f'__ All shape: {df.shape}')
  #df.to_csv(api.vars['varini'][api.vars['varini']['env']]['merged2'] + 'group.csv')


###Version:
05/11 08:28
ready to import packages

from censustools import *

##MAIN function

In [ ]:
api2=API('gdrive')
#unMergedPath = api.vars['varini'][api.vars['varini']['env']]['unMergedPath']
single_file_generator_01(api2)
merge_02(api2)
group_by_03(api2)
api2.df = readCSV_and_fuse_dfs_04()

INFO:activity_logger:Activity: allMonthList3TLA
INFO:activity_logger:Activity: allMonthList3TLA
INFO:activity_logger:Activity: allMonthList3TLA
INFO:activity_logger:Activity: allMonthList3TLA
INFO:activity_logger:Activity: allMonthList3TLA
INFO:activity_logger:Activity: allMonthList3TLA
INFO:activity_logger:Activity: allMonthList3TLA
INFO:activity_logger:Activity: allMonthList3TLA
INFO:activity_logger:Activity: allMonthList3TLA
INFO:activity_logger:Activity: allMonthList3TLA
INFO:activity_logger:Activity: allMonthList3TLA
INFO:activity_logger:Activity: allMonthList3TLA
INFO:activity_logger:Activity: allMonthList3TLA
INFO:activity_logger:Activity: allMonthList3TLA
INFO:activity_logger:Activity: allMonthList3TLA
INFO:activity_logger:> vvvvvvvvvvvvvvvvvvvvvvvvvvvvvv year:2024,month:jun vvvvvvvvvvvvvvvvvvvvvvvvvvvvvv
INFO:activity_logger:Class MixComp: / main >  method - Entrance
INFO:activity_logger:>Class MixComp: / main > get() method
INFO:activity_logger:Activity: getVarsNameFromUrl
IN

Begin> {'initiallyInFile': 14, 'processed': 0, 'skipped': 0}
checkVariable() = {'PWSSWGT': True, 'PEMNTVTY': True, 'PEMARITL': True, 'PESEX': True, 'PRCITSHP': True, 'PTIO': False, 'OCD': False, 'PECYC': True, 'PEDIPGED': True, 'GTCBSA': False, 'status': False}


INFO:activity_logger:Class MixComp: / get() / > Return dict: [[[[ {'response time in secs': 0, 'status': False, 'code': 404, 'comment': 'url:https://api.census.gov/data/2024/cps/basic/jun?get=PWSSWGT,PEMNTVTY,PEMARITL,PESEX,PRCITSHP,PTIO1OCD,PTIO2OCD,PECYC,PEDIPGED,CBSA&for=state:*&key=a24a131b4da25b83199bdc7202fcab553b9e2983'} ]]]]
INFO:activity_logger:Class MixComp: / main - line: get() method - Result:{'response time in secs': 0, 'status': False, 'code': 404, 'comment': 'url:https://api.census.gov/data/2024/cps/basic/jun?get=PWSSWGT,PEMNTVTY,PEMARITL,PESEX,PRCITSHP,PTIO1OCD,PTIO2OCD,PECYC,PEDIPGED,CBSA&for=state:*&key=a24a131b4da25b83199bdc7202fcab553b9e2983'}
INFO:activity_logger:Class MixComp: / main > Error during URL call, Code return: 404, Skipped !!
INFO:activity_logger:Class MixComp: / main > Called main(self) method - End
INFO:activity_logger:> ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


Function: 'get()'. Census API query took: 0.24506115913391113 seconds

End  < {'initiallyInFile': 14, 'processed': 1, 'skipped': 14}
new_census_data: False, count['processed']: 1,count['skipped']: 14


In [ ]:
api2.df.head()

,Unnamed: 0,state,nativity,marital,sex,citiz,occ1,occ2,collegcred,highsch,city,YYYYMM,weight
0,0,1,57,1,1,1,-1,-1,2,1,13820,2010-06-01,21234.0041
1,1,1,57,1,1,1,-1,-1,2,1,19460,2010-06-01,6152.8846
2,2,1,57,1,1,1,-1,-1,2,1,33660,2010-06-01,6301.1145
3,3,1,57,1,1,1,-1,-1,2,1,33860,2010-06-01,9017.3094
4,4,1,57,1,1,1,-1,-1,2,1,46220,2010-06-01,2770.9996


##Section 2: Graphics (Streamlit, ...)

###Attempt to display graph

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.5 MB/s eta 0:00:00


In [ ]:
import streamlit as st

In [ ]:
pd.unique(df['YYYYMM'])

array(['2010-06-01', '2011-06-01', '2012-06-01', '2013-06-01',
       '2014-06-01', '2015-06-01', '2016-06-01', '2017-06-01',
       '2018-06-01', '2019-06-01', '2020-06-01', '2021-06-01',
       '2022-06-01', '2023-06-01'], dtype=object)

In [ ]:
"""
choose_year     = pd.to_datetime(df['YYYYMM']).dt.year==2010
choose_state    = df['state']==10
choose_month    = pd.to_datetime(df['YYYYMM']).dt.month==5
choose_marital  = None
choose_citiz    = None
choose_collegcred = None
choose_highsch  = None
"""
choose_year     = None
choose_year     = pd.to_datetime(df['YYYYMM']).dt.year==2010
choose_state    = None
choose_state    = df['state']==20
choose_marital  = None
choose_marital  = df['marital']== 1
choose_citiz    = None
choose_citiz    = df['citiz']== 1
choose_collegcred = None
choose_collegcred = df['collegcred']== 1
choose_highsch  = None
choose_highsch  = df['highsch']== 2

condition = None

if choose_year is not None:
  if condition is None: condition = choose_year
  else: condition &= choose_year

if choose_state is not None:
  if condition is None: condition = choose_state
  else: condition &= choose_state

if choose_marital is not None:
  if condition is None: condition = choose_marital
  else: condition &= choose_marital

if choose_citiz is not None:
  if condition is None: condition = choose_citiz
  else: condition &= choose_citiz

if choose_collegcred is not None:
  if condition is None: condition = choose_collegcred
  else: condition &= choose_collegcred


if choose_highsch is not None:
  if condition is None: condition = choose_highsch
  else: condition &= choose_highsch

#if none selected display everything
if condition is None: condition=pd.Series([True] * len(df))

#g2 = df [(pd.to_datetime(df['YYYYMM']).dt.month==5) & (df['state']==10)]
g2 = df [condition]
#Display the 7 highest population
g2 = g2.groupby(['state'])['weight'].sum().reset_index().sort_values(by='weight', ascending=False).head(7)
g2['state'] = g2['state'].apply(lambda x: api.id_to_stateName(x))
print (g2)

   state       weight
2     CA  266845.4470
40    TX   73669.0105
44    WA   59468.8229
7     FL   56694.4953
30    NY   43190.9496
20    MI   35991.8432
31    NC   32821.7367
